In [28]:
import sympy as sp
from IPython.display import display

In [3]:
x = sp.Symbol('x')
expression = 2*(x/2)**2
a = sp.diff(expression, x)
a

x

Metric

In [4]:
n = 4
coordinates = []
for i in range(n):
    coordinate = sp.Symbol(input(f'Enter the coordinate {i}'))
    coordinates.append(coordinate)
coordinates_sym = sp.Matrix([coordinates])
coordinates_sym

Enter the coordinate 0 t
Enter the coordinate 1 r
Enter the coordinate 2 \theta
Enter the coordinate 3 \phi


Matrix([[t, r, \theta, \phi]])

In [5]:
n = 4
t = sp.Symbol('t')
r = sp.Symbol('r')
theta = sp.Symbol('\\theta')
phi = sp.Symbol('\\phi')

In [66]:
# No run
symbol = '-x^2/(x+2)^3'
array = ['+', '-', '*', '/', '^', 'sin', 'cos', 'tan', 'ctg', 'sec', 'csc']
if set(symbol) & set(array):
    # Power
    if '^' in symbol:
        # Find the indexs
        power_indexs = []
        for i in range(len(symbol)):
            if symbol[i] == '^':
                power_indexs.append(i)
                for j in range(len(symbol.count('^'))):
                    if (symbol[power_indexs[j]+1] != '('):
                        exponent = symbol[power_indexs[j]+1]
        
else:
    symbol = sp.Symbol(symbol)
print(power_indexs)

[2, 10]


12

In [136]:
# No run
is_symmetrical = True
g = []
for i in range(n):
    g_row = []
    for j in range(n):
        g_ij = input(f'Enter the component g_{i}{j}: ')
        try:
            float(g_ij)
            is_number = True
        except:
            is_number = False
        if (is_number == False):
            if '-' in g_ij:
                g_ij = -sp.Symbol(g_ij.replace('-',''))
            
            else:
                g_ij = sp.Symbol(g_ij)
        g_row.append(g_ij)
    g.append(g_row)
g
g_sym = sp.Matrix(g)
g_sym

Enter the component g_00:  -r
Enter the component g_01:  0
Enter the component g_10:  0
Enter the component g_11:  r


Matrix([
[-r, 0],
[ 0, r]])

In [6]:
f = sp.Function('f')(r)
g = [[-f, 0, 0, 0],
    [0, 1/f, 0, 0],
    [0, 0, r**2, 0],
    [0, 0, 0, r**2*sp.sin(theta)**2]]
g_sym = sp.Matrix(g)
g_sym

Matrix([
[-f(r),      0,    0,                   0],
[    0, 1/f(r),    0,                   0],
[    0,      0, r**2,                   0],
[    0,      0,    0, r**2*sin(\theta)**2]])

In [7]:
g_inverse = g_sym.inv()
g_inverse

Matrix([
[-1/f(r),    0,       0,                       0],
[      0, f(r),       0,                       0],
[      0,    0, r**(-2),                       0],
[      0,    0,       0, 1/(r**2*sin(\theta)**2)]])

Computing of Christofell symbols

In [15]:
def christoffel_symbol(lmbd,mu,nu):
    g_component = lambda nu, sigma: sp.Symbol(f'g_{{{nu}{sigma}}}')
    g_component_up = lambda nu, sigma: sp.Symbol(f'g^{{{nu}{sigma}}}')
    christoffel_component_1 = lambda mu, nu: sp.Derivative(g_component(nu, sp.Symbol('\\sigma')), coordinates_sym[mu])
    christoffel_component_2 = lambda mu, nu: sp.Derivative(g_component(mu, sp.Symbol('\\sigma')), coordinates_sym[nu])
    christoffel_component_3 = lambda mu, nu: sp.Derivative(g_component(mu, nu), sp.Symbol('x^\\sigma'))
    return sp.Symbol(f'\\Gamma^{lmbd}_{{{mu}{nu}}} =  \\frac{1}{2}')*g_component_up(lmbd, sp.Symbol('\\sigma'))*(christoffel_component_1(mu,nu) + christoffel_component_2(mu,nu) - christoffel_component_3(mu,nu))
christoffel_symbol(1,0,1)

\Gamma^1_{01} =  \frac12*g^{1\sigma}*(-Derivative(g_{01}, x^\sigma) + Derivative(g_{0\sigma}, r) + Derivative(g_{1\sigma}, t))

In [12]:
def christoffel_symbol_computing(lmbd,mu,nu):
    index = lambda i,j: i*n + j
    sum = 0
    for i in range(n):
        sum = sum + g_inverse[index(lmbd,i)]*( sp.diff(g_sym[index(nu,i)], coordinates_sym[mu]) + sp.diff(g_sym[index(mu,i)], coordinates_sym[nu]) - sp.diff(g_sym[index(mu,nu)], coordinates_sym[i]) )/2
    return sum

In [109]:
for i in range(n):
    for j in range(n):
        for k in range(n):
            christoffel_component = christoffel_symbol_computing(i,j,k)
            if christoffel_component != 0:
                gamma = sp.Symbol(f'\\Gamma^{i}_{{{j}{k}}}')
                display(sp.Eq(sp.Symbol(f'\\Gamma^{i}_{{{j}{k}}}'), christoffel_component))

Eq(\Gamma^0_{01}, Derivative(f(r), r)/(2*f(r)))

Eq(\Gamma^0_{10}, Derivative(f(r), r)/(2*f(r)))

Eq(\Gamma^1_{00}, f(r)*Derivative(f(r), r)/2)

Eq(\Gamma^1_{11}, -Derivative(f(r), r)/(2*f(r)))

Eq(\Gamma^1_{22}, -r*f(r))

...

Eq(\Gamma^2_{33}, -sin(\theta)*cos(\theta))

Eq(\Gamma^3_{13}, 1/r)

Eq(\Gamma^3_{23}, cos(\theta)/sin(\theta))

Eq(\Gamma^3_{31}, 1/r)

Eq(\Gamma^3_{32}, cos(\theta)/sin(\theta))

Computing of Riemann tensor $R^{\mu}_{\nu\rho\sigma}$

In [97]:
def riemann_tensor_computing(mu, nu, rho, sigma):
    riemann_tensor_1 = sp.diff(christoffel_symbol_computing(mu,nu,sigma), coordinates_sym[rho])
    riemann_tensor_2 = sp.diff(christoffel_symbol_computing(mu,nu,rho), coordinates_sym[sigma])
    riemann_tensor_3 = 0
    for lmbd in range(n):
        riemann_tensor_3 = riemann_tensor_3 + christoffel_symbol_computing(mu,rho,lmbd)*christoffel_symbol_computing(lmbd,nu,sigma)
    riemann_tensor_4 = 0
    for lmbd in range(n):
        riemann_tensor_4 = riemann_tensor_4 + christoffel_symbol_computing(mu,sigma,lmbd)*christoffel_symbol_computing(lmbd,nu,rho)
    return riemann_tensor_1 - riemann_tensor_2 + riemann_tensor_3 - riemann_tensor_4

In [110]:
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                riemann_component = riemann_tensor_computing(i,j,k,l)
                if riemann_component != 0:
                    Riemann_symbol = sp.Symbol(f'R^{i}_{{{j}{k}{l}}}')
                    display(sp.Eq(Riemann_symbol,riemann_component))

Eq(R^0_{101}, -Derivative(f(r), (r, 2))/(2*f(r)))

Eq(R^0_{110}, Derivative(f(r), (r, 2))/(2*f(r)))

Eq(R^0_{202}, -r*Derivative(f(r), r)/2)

Eq(R^0_{220}, r*Derivative(f(r), r)/2)

Eq(R^0_{303}, -r*sin(\theta)**2*Derivative(f(r), r)/2)

...

Eq(R^3_{030}, f(r)*Derivative(f(r), r)/(2*r))

Eq(R^3_{113}, Derivative(f(r), r)/(2*r*f(r)))

Eq(R^3_{131}, -Derivative(f(r), r)/(2*r*f(r)))

Eq(R^3_{223}, f(r) - 1)

Eq(R^3_{232}, 1 - f(r))